# Bag Of Words sur le Recueil

In [1]:
cd ../..

/Users/fm/Desktop/Work/Patat


In [2]:
import pandas as pd

## Chargement Corpus 
---

In [3]:
corpus_filename = '220930-RecueilComplet.csv'

In [4]:
df_corpus = pd.read_csv(f'data/corpus/{corpus_filename}')

In [5]:
df_corpus = df_corpus[df_corpus['parse_method']=='parse']

In [6]:
df_corpus

,url,title,article,parse_method,date,tag,abstract,author,date_iso,words,...,des pages et des pages,des chiffres et des tableaux,sophisme,inversion de la preuve,invérifiable,propos rapporté,sources citées,titre décalé,site,text
2,https://www.breizh-info.com/2022/02/23/180457/...,Tests – vaccins – pass : fabrique de la paniqu...,Cela fait 2 ans que la crise du coronavirus a ...,parse,Le \n\t\t\t\t\t\t\t\t\t\t23 février 2022,"['Santé', 'covid', 'crise', ""décoder l'éco"", '...",NaN,NaN,2022-02-23T00:00:00,NaN,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,www.breizh-info.com,Tests – vaccins – pass : fabrique de la paniqu...
3,https://www.breizh-info.com/2022/02/20/180182/...,"Covid-19. La pandémie et les vaccins, pour mie...","Ces dernières semaines, le monde entier a prêt...",parse,Le \n\t\t\t\t\t\t\t\t\t\t20 février 2022,"['A La Une', 'Santé', ""coup d'etat mondial"", '...",NaN,NaN,2022-02-20T00:00:00,NaN,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,www.breizh-info.com,"Covid-19. La pandémie et les vaccins, pour mie..."
4,https://www.breizh-info.com/2022/04/19/185260/...,Covid-19. Une épidémie quasiment terminée avan...,Voici mon avant dernier bulletin de situation ...,parse,Le \n\t\t\t\t\t\t\t\t\t\t19 avril 2022,"['A La Une', 'Santé', 'COVID-19', 'epidemie', ...",NaN,NaN,2022-04-19T00:00:00,NaN,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,www.breizh-info.com,Covid-19. Une épidémie quasiment terminée avan...
5,https://www.breizh-info.com/2022/04/15/184984/...,"Piscine, voile, ramadan : scènes de la vie ord...",On a beaucoup parlé du Grand remplacement dura...,parse,Le \n\t\t\t\t\t\t\t\t\t\t15 avril 2022,"['Education', 'Société', 'burkini', 'hallal', ...",NaN,NaN,2022-04-15T00:00:00,NaN,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,www.breizh-info.com,"Piscine, voile, ramadan : scènes de la vie ord..."
6,https://www.breizh-info.com/2021/11/02/173738/...,Faut-il vous métisser comme le suggère la pres...,Comme nous le verrons dans l’ensemble des étud...,parse,Le \n\t\t\t\t\t\t\t\t\t\t2 novembre 2021,"['Sciences', 'métis', 'metissage', 'race']",NaN,NaN,2021-11-02T00:00:00,NaN,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,www.breizh-info.com,Faut-il vous métisser comme le suggère la pres...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,https://www.wikistrike.com/2022/08/fresne-cett...,Fresnes: cette prison où les condamnés finisse...,Cette photo est une erreur \n\nEt nul n'est au...,parse,Publié par wikistrike.com\n ...,[],NaN,NaN,2022-08-21T00:00:00,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,www.wikistrike.com,Fresnes: cette prison où les condamnés finisse...
464,https://www.wikistrike.com/2022/08/le-forum-ec...,Le Forum économique mondial estime qu’il exist...,Ne trahissant pas sa réputation de méchant d’u...,parse,Publié par wikistrike.com\n ...,[],NaN,NaN,2022-08-23T00:00:00,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,www.wikistrike.com,Le Forum économique mondial estime qu’il exist...
465,https://www.wikistrike.com/2022/08/le-masque-r...,"Le masque réduit les infections COVID de 0,06%",par Collectif Reinfocovid.\nCet article illust...,parse,Publié par wikistrike.com\n ...,[],NaN,NaN,2022-08-24T00:00:00,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,www.wikistrike.com,"Le masque réduit les infections COVID de 0,06%..."
466,https://www.wikistrike.com/2022/08/l-un-des-pl...,L’un des plus grands complexes mégalithiques d...,"En Espagne, le site d’une future plantation d’...",parse,Publié par wikistrike.com\n ...,[],NaN,NaN,2022-08-24T00:00:00,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,www.wikistrike.com,L’un des plus grands complexes mégalithiques d...


## Mots non signifiants
---

In [7]:
import patat.ml.lex_analyser

lex = patat.ml.lex_analyser.LexAnalyser()

def merge_texts(texts):
    result = ''
    for text in texts:
        result = result + text + '\n'
    return result

def count_words_label(df,label):
    df_texts = pd.pivot_table(df, values='text', index=None, columns=label, aggfunc=merge_texts)
    wc = {}
    word_analysis = {}
    for key in df_texts.keys():
        text = df_texts[key]['text']
        count_colname = label+'_'+str(key)
        wc[key] = lex.count_tokens(lex.get_words(text))
        for word in wc[key]:
            word_dic = word_analysis.get(word,{})
            word_dic[count_colname]=wc[key][word]
            word_analysis[word]=word_dic
    return word_analysis

In [8]:
df_count = pd.DataFrame(count_words_label(df_corpus,'infox')).T
df_count = df_count.fillna(0)

In [9]:
df_count.shape

(34238, 2)

In [10]:
df_count

,infox_0.0,infox_1.0
rennes,24.0,6.0
reprise,10.0,8.0
des,3173.0,6186.0
vols,17.0,13.0
commerciaux,12.0,7.0
...,...,...
rogov,0.0,1.0
dniepropietrovsk,0.0,1.0
energodar,0.0,1.0
barges,0.0,1.0


### Identification des mots rares

In [11]:
occ_rare = 3
def is_rare(row):
    return row['infox_0.0'] < occ_rare and row['infox_1.0'] < occ_rare

df_rare = df_count[df_count.apply(is_rare,axis=1)]

In [12]:
rare_words = list(df_rare.index)
len(rare_words)

22646

### Identification des mots communs

In [13]:
common_size = 100
top_0 = df_count.sort_values('infox_0.0',ascending=False).head(common_size).index
top_1 = df_count.sort_values('infox_1.0',ascending=False).head(common_size).index
common_words = []
for word in top_0:
    if word in top_1:
        common_words.append(word)
len(common_words)

88

In [14]:
common_words

['de',
 'la',
 'le',
 'les',
 'et',
 'des',
 'en',
 'un',
 'du',
 'est',
 'une',
 'que',
 'pour',
 'dans',
 'qui',
 'il',
 'par',
 'sur',
 'au',
 'pas',
 'plus',
 'ce',
 'ne',
 'qu',
 'se',
 'ont',
 'avec',
 'sont',
 'ou',
 'son',
 'été',
 'nous',
 'aux',
 'mais',
 'comme',
 'cette',
 'on',
 'pays',
 'elle',
 'fait',
 'être',
 'tout',
 'ces',
 'même',
 'leur',
 'aussi',
 'sa',
 'si',
 'monde',
 'ses',
 'contre',
 'sans',
 'bien',
 'vous',
 'ils',
 'faire',
 'depuis',
 'je',
 'russie',
 'deux',
 'après',
 'entre',
 'avait',
 'guerre',
 'non',
 'ukraine',
 'autres',
 'peut',
 'covid',
 'était',
 'selon',
 'avoir',
 'ans',
 'tous',
 'cela',
 'où',
 'alors',
 'leurs',
 'encore',
 'très',
 'moins',
 'dont',
 'états',
 'autre',
 'également',
 'état',
 'cas',
 'soit']

In [15]:
ignore_words = common_words + rare_words

## Count Vectorizer
---

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
count_vectorizer = CountVectorizer(stop_words=ignore_words)
#count_vectorizer = CountVectorizer()

In [18]:
corpus = df_corpus['text']
X = count_vectorizer.fit_transform(corpus)
y = df_corpus['infox']

### Construction des Datasets d'entrainement et de test

In [44]:
import sklearn.model_selection

#X_train,X_test,y_train,y_test = sklearn.model_selection.train_test_split(X,y,train_size=0.8,shuffle=True)
X_train,X_test,y_train,y_test = sklearn.model_selection.train_test_split(X,y)

In [45]:
X_train.shape[0]

332

### Entrainement du modele

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

#lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr', max_iter=1000)
cv_lr = LogisticRegression(max_iter=1000)

# Fit the model
cv_lr.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

### Prédictions

In [50]:
y_pred = cv_lr.predict(X_test)

### Mesure performances

In [51]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
print('Matrice de confusion')
print(cnf_matrix)
print(f'Accuracy score {metrics.accuracy_score(y_test, y_pred)*100:.2f}%')
print(f'Recall score {metrics.recall_score(y_test, y_pred)*100:.2f}%')

Matrice de confusion
[[44 11]
 [15 41]]
Accuracy score 76.58%
Recall score 73.21%


## Tf/Idf Vectorizer
---

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
#tfidf_vectorizer = TfidfVectorizer(stop_words=ignore_words)
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit_transform(corpus)

<443x35703 sparse matrix of type '<class 'numpy.float64'>'
	with 197956 stored elements in Compressed Sparse Row format>

In [35]:
corpus = df_corpus['text']
X = count_vectorizer.fit_transform(corpus)
y = df_corpus['infox']

In [40]:
### Construction des Datasets d'entrainement et de test

import sklearn.model_selection

#X_train,X_test,y_train,y_test = sklearn.model_selection.train_test_split(X,y,train_size=0.6,shuffle=True)
X_train,X_test,y_train,y_test = sklearn.model_selection.train_test_split(X,y)

X_train.shape[0]/X.shape[0]

0.7494356659142212

In [53]:

#lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr', max_iter=1000)
tf_lr = LogisticRegression(max_iter=1000)

# Fit the model
tf_lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

### Prédictions

In [54]:
y_pred = tf_lr.predict(X_test)

### Mesure des résultats

In [55]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
print('Matrice de confusion')
print(cnf_matrix)
print(f'Accuracy score {metrics.accuracy_score(y_test, y_pred)*100:.2f}%')
print(f'Recall score {metrics.recall_score(y_test, y_pred)*100:.2f}%')

Matrice de confusion
[[44 11]
 [15 41]]
Accuracy score 76.58%
Recall score 73.21%


## Sauvegarde des predicteurs
---

In [61]:
import time
import pickle

In [68]:
def save_predictor(descriptor,classifier,data='',filename=''):
    predictor = {
        'descriptor' : descriptor,
        'classifier' : classifier,
        'data' : data
    }
    if filename == '':
        filename = f'data/predictors/{str(int(time.time()))}.pp'
    with open(filename, 'wb') as file:
        pickle.dump(predictor, file)

In [69]:
save_predictor(tfidf_vectorizer,tf_lr,filename='data/predictors/220930-tf_lr.pp')

In [70]:
save_predictor(count_vectorizer,cv_lr,filename='data/predictors/220930-cv_lr.pp')

## Bricolages
---

In [57]:
type(tf_lr)

sklearn.linear_model._logistic.LogisticRegression

In [58]:
import time

In [60]:
int(time.time())

1664570915